In [1]:
#ใส่ User Agent https://www.whatismybrowser.com/detect/what-is-my-user-agent/#google_vignette

head = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'

import warnings
warnings.filterwarnings("ignore")

In [2]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd

underlying = ['AWC', 'BANPU', 'BGRIM', 'BTS', 'CBG', 'COM7', 'DELTA', 'EA', 'GLOBAL', 'GPSC', 'IVL', 'KCE']
tv = TvDatafeed()

df = tv.get_hist(symbol='SET:SET50', exchange='SET', interval=Interval.in_daily, n_bars=1100)
df_und = df[['close']]
df_und.rename(columns = {'close':'remove'}, inplace=True)

for i in range(len(underlying)):
    try:
        df = tv.get_hist(symbol='SET:' + underlying[i], exchange='SET', interval=Interval.in_daily, n_bars=1100)
        df = df[['close']]
        df_und = pd.merge(df_und, df, left_index=True, right_index=True, how='outer')
        df_und.rename(columns = {'close':'SET:' + underlying[i]}, inplace=True)
    except:
        df = tv.get_hist(symbol='SET:' + underlying[i], exchange='SET', interval=Interval.in_daily, n_bars=1100)
        df = df[['close']]
        df_und = pd.merge(df_und, df, left_index=True, right_index=True, how='outer')
        df_und.rename(columns = {'close':'SET:' + underlying[i]}, inplace=True)
df_und = df_und.drop('remove',axis=1)

you are using nologin method, data you access may be limited


S-BULL ELN

In [3]:
import numpy as np
df_vol = pd.DataFrame()
df_vol['Underlying'] = np.array(underlying)

#risk free
from tvDatafeed import TvDatafeed, Interval
tv = TvDatafeed()
rf = tv.get_hist(symbol='ECONOMICS:THINTR', exchange='Central Bank',interval=Interval.in_monthly,n_bars=2)
rrf = rf['close'][-1]
rrf = rrf/100

ttm = 15
strikeLV1 = 98
strikeLV2 = 97
strikeLV3 = 96

spot = []
for col in df_und.columns:
    last_value = df_und[col].iloc[-1]
    spot.append(last_value)
df_vol['S'] = np.array(spot)
df_vol['K1'] = df_vol['S'] * strikeLV1/100
df_vol['K2'] = df_vol['S'] * strikeLV2/100
df_vol['K3'] = df_vol['S'] * strikeLV3/100

df_vol = pd.melt(df_vol, id_vars=['Underlying', 'S'], value_vars=['K1', 'K2', 'K3'], var_name='K')
# Sort the DataFrame by 'Underlying' and 'S'
df_vol.sort_values(by=['Underlying', 'S'], inplace=True)
# Reset index
df_vol.reset_index(drop=True, inplace=True)
df_vol = df_vol.drop(columns='K',axis=1)
df_vol.rename(columns={'value' : 'K'},inplace=True)
level = []
for i in range(len(df_vol)):
    if i % 3 == 0:
        strlv = strikeLV1
    elif i % 3 == 1:
        strlv = strikeLV2
    else:
        strlv = strikeLV3
    level.append(strlv)
df_vol['Strike Level (%)'] = np.array(level)

Garch

In [4]:
import numpy as np
returns = np.log(df_und).diff()*100
from arch import arch_model

volatility_list = []
for i in range (len(df_vol)):
    returns_each = returns[['SET:' +df_vol['Underlying'][i]]].dropna()
    model = arch_model(returns_each,vol="Garch", p=1,q=1).fit(disp='off')
    forecast = model.forecast(horizon = ttm, method = 'analytic', reindex=True)

    y = forecast.variance.values[-1,:].flatten()
    df_y = pd.DataFrame(y)
    df_y.rename(columns={df_y.columns[0]:'forecast'},inplace=True)
    df_y['forecast'] = np.sqrt(df_y['forecast']) * np.sqrt(250)
    vol = float(df_y['forecast'].tail(1))
    volatility_list.append(vol)
df_vol['Forecasted Vol.'] = np.array(volatility_list)

In [5]:
hist = []
for i in range(len(df_vol)):
   std = returns['SET:'+df_vol['Underlying'][i]][-ttm:].std() * np.sqrt(250)
   hist.append(std)
df_vol['Hist Vol.'] = np.array(hist)

#Black Scholes Template

from scipy.stats import norm
import numpy as np
p = []
for i in range(len(df_vol)):
    d1 = (np.log(df_vol['S'][i]/df_vol['K'][i])+((rrf+(((df_vol['Forecasted Vol.'][i]/100)**2)/2))*(ttm/365)))/((df_vol['Forecasted Vol.'][i]/100)*np.sqrt(ttm/365))
    d2 = d1 - ((df_vol['Forecasted Vol.'][i]/100)*np.sqrt(ttm/365))

    y = (df_vol['K'][i] * np.exp(-rrf * (ttm/365)) * norm.cdf(-d2,0,1)) - (df_vol['S'][i] * norm.cdf(-d1,0,1))

    p.append(y)

df_vol['Put'] = np.array(p)

df_vol

df_vol['Issue Price (%)'] = (1-(df_vol['Put']/df_vol['K'])) * 100
df_vol['WHT (%)'] = ((df_vol['Put']) / df_vol['K']) * 0.15 * 100
df_vol['Net Settlement Including WHT(%)'] = df_vol['Issue Price (%)'] + df_vol['WHT (%)']
df_vol['Annualized Yield (Before Tax)'] = df_vol['Put']/df_vol['K'] * 365 /15
df_sbull = df_vol.copy()
df_sbull

,Underlying,S,K,Strike Level (%),Forecasted Vol.,Hist Vol.,Put,Issue Price (%),WHT (%),Net Settlement Including WHT(%),Annualized Yield (Before Tax)
0,AWC,3.82,3.7436,98,33.779307,28.184116,0.068039,98.182519,0.272622,98.455141,0.442254
1,AWC,3.82,3.7054,97,33.779307,28.184116,0.054184,98.537712,0.219343,98.757056,0.355823
2,AWC,3.82,3.6672,96,33.779307,28.184116,0.042418,98.843314,0.173503,99.016817,0.281460
3,BANPU,5.40,5.2920,98,37.688437,39.052670,0.112350,97.876983,0.318453,98.195435,0.516601
4,BANPU,5.40,5.2380,97,37.688437,39.052670,0.091871,98.246067,0.263090,98.509157,0.426790
5,BANPU,5.40,5.1840,96,37.688437,39.052670,0.074096,98.570681,0.214398,98.785079,0.347801
6,BGRIM,24.80,24.3040,98,30.876324,27.039630,0.387091,98.407296,0.238906,98.646201,0.387558
7,BGRIM,24.80,24.0560,97,30.876324,27.039630,0.300739,98.749838,0.187524,98.937362,0.304206
8,BGRIM,24.80,23.8080,96,30.876324,27.039630,0.228923,99.038460,0.144231,99.182691,0.233975
9,BTS,5.05,4.9490,98,75.942442,62.217379,0.256463,94.817885,0.777317,95.595202,1.260981


Bull-P ELN

In [6]:
df_vol = pd.DataFrame()
df_vol['Underlying'] = np.array(underlying)

ttm = 30
protectLV = 80
strikeLV1 = 98
strikeLV2 = 97
strikeLV3 = 96

spot = []
for col in df_und.columns:
    last_value = df_und[col].iloc[-1]
    spot.append(last_value)
df_vol['S'] = np.array(spot)
df_vol['K1'] = df_vol['S'] * strikeLV1/100
df_vol['K2'] = df_vol['S'] * strikeLV2/100
df_vol['K3'] = df_vol['S'] * strikeLV3/100

df_vol = pd.melt(df_vol, id_vars=['Underlying', 'S'], value_vars=['K1', 'K2', 'K3'], var_name='K')
# Sort the DataFrame by 'Underlying' and 'S'
df_vol.sort_values(by=['Underlying', 'S'], inplace=True)
# Reset index
df_vol.reset_index(drop=True, inplace=True)
df_vol = df_vol.drop(columns='K',axis=1)
df_vol.rename(columns={'value' : 'K'},inplace=True)
level = []
for i in range(len(df_vol)):
    if i % 3 == 0:
        strlv = strikeLV1
    elif i % 3 == 1:
        strlv = strikeLV2
    else:
        strlv = strikeLV3
    level.append(strlv)
df_vol['Strike Level (%)'] = np.array(level)
df_vol['Protection Price'] = df_vol['K'] * protectLV/100

In [7]:
volatility_list = []
for i in range (len(df_vol)):
    returns_each = returns[['SET:' +df_vol['Underlying'][i]]].dropna()
    model = arch_model(returns_each,vol="Garch", p=1,q=1).fit(disp='off')
    forecast = model.forecast(horizon = ttm, method = 'analytic', reindex=True)

    y = forecast.variance.values[-1,:].flatten()
    df_y = pd.DataFrame(y)
    df_y.rename(columns={df_y.columns[0]:'forecast'},inplace=True)
    df_y['forecast'] = np.sqrt(df_y['forecast']) * np.sqrt(250)
    vol = float(df_y['forecast'].tail(1))
    volatility_list.append(vol)
df_vol['Forecasted Vol.'] = np.array(volatility_list)

In [8]:
hist = []
for i in range(len(df_vol)):
   std = returns['SET:'+df_vol['Underlying'][i]][-ttm:].std() * np.sqrt(250)
   hist.append(std)
df_vol['Hist Vol.'] = np.array(hist)

#Black Scholes Template

p = []
for i in range(len(df_vol)):
    d1 = (np.log(df_vol['S'][i]/df_vol['K'][i])+((rrf+(((df_vol['Forecasted Vol.'][i]/100)**2)/2))*(ttm/365)))/((df_vol['Forecasted Vol.'][i]/100)*np.sqrt(ttm/365))
    d2 = d1 - ((df_vol['Forecasted Vol.'][i]/100)*np.sqrt(ttm/365))

    y = (df_vol['K'][i] * np.exp(-rrf * (ttm/365)) * norm.cdf(-d2,0,1)) - (df_vol['S'][i] * norm.cdf(-d1,0,1))

    p.append(y)

df_vol['Put'] = np.array(p)

p = []
for i in range(len(df_vol)):
    d1 = (np.log(df_vol['S'][i]/df_vol['Protection Price'][i])+((rrf+(((df_vol['Forecasted Vol.'][i]/100)**2)/2))*(ttm/365)))/((df_vol['Forecasted Vol.'][i]/100)*np.sqrt(ttm/365))
    d2 = d1 - ((df_vol['Forecasted Vol.'][i]/100)*np.sqrt(ttm/365))

    y = (df_vol['Protection Price'][i] * np.exp(-rrf * (ttm/365)) * norm.cdf(-d2,0,1)) - (df_vol['S'][i] * norm.cdf(-d1,0,1))
    p.append(y)
df_vol['Put(LP)'] = np.array(p)

df_vol['Total'] = df_vol['Put'] - df_vol['Put(LP)']

df_vol['Issue Price (%)'] = (1-(df_vol['Total']/df_vol['K'])) * 100
df_vol['WHT (%)'] = ((df_vol['Total']) / df_vol['K']) * 0.15 * 100
df_vol['Net Settlement Including WHT(%)'] = df_vol['Issue Price (%)'] + df_vol['WHT (%)']
df_vol['Annualized Yield (Before Tax)'] = df_vol['Total']/df_vol['K'] * 365 /ttm


df_bullp = df_vol.copy()
df_bullp

,Underlying,S,K,Strike Level (%),Protection Price,Forecasted Vol.,Hist Vol.,Put,Put(LP),Total,Issue Price (%),WHT (%),Net Settlement Including WHT(%),Annualized Yield (Before Tax)
0,AWC,3.82,3.7436,98,2.99488,35.494001,30.028264,0.114909,0.000902,0.114007,96.954608,0.456809,97.411417,0.370523
1,AWC,3.82,3.7054,97,2.96432,35.494001,30.028264,0.099139,0.000659,0.098480,97.342255,0.398662,97.740916,0.323359
2,AWC,3.82,3.6672,96,2.93376,35.494001,30.028264,0.084854,0.000476,0.084378,97.699116,0.345133,98.044249,0.279941
3,BANPU,5.40,5.2920,98,4.23360,38.837125,34.983684,0.182387,0.002566,0.179820,96.602032,0.509695,97.111727,0.413419
4,BANPU,5.40,5.2380,97,4.19040,38.837125,34.983684,0.159505,0.001957,0.157549,96.992200,0.451170,97.443370,0.365949
5,BANPU,5.40,5.1840,96,4.14720,38.837125,34.983684,0.138559,0.001477,0.137081,97.355686,0.396647,97.752333,0.321725
6,BGRIM,24.80,24.3040,98,19.44320,31.774114,32.779488,0.644533,0.002159,0.642375,97.356918,0.396462,97.753380,0.321575
7,BGRIM,24.80,24.0560,97,19.24480,31.774114,32.779488,0.545681,0.001479,0.544202,97.737769,0.339335,98.077103,0.275238
8,BGRIM,24.80,23.8080,96,19.04640,31.774114,32.779488,0.457463,0.000998,0.456465,98.082722,0.287592,98.370314,0.233269
9,BTS,5.05,4.9490,98,3.95920,65.936261,48.883362,0.323103,0.038473,0.284630,94.248730,0.862691,95.111420,0.699738


In [9]:
from datetime import datetime

dataframes = {
    'S-BULL': df_sbull,
    'BULL-P': df_bullp,

}

# Get today's date in the format yyyy-mm-dd
today_date = datetime.today().strftime('%Y-%m-%d')

# Specify the output file name with today's date
output_file = f'ELN._{today_date}.xlsx'

# Use ExcelWriter to save each DataFrame to a separate worksheet
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for sheet_name, dataframe in dataframes.items():
        dataframe.to_excel(writer, sheet_name=sheet_name, index=False)
            